# --- Day 7: Amplification Circuit ---
https://adventofcode.com/2019/day/7

Based on the navigational maps, you're going to need to send more power to your ship's thrusters to reach Santa in time. To do this, you'll need to configure a series of amplifiers already installed on the ship.

There are five amplifiers connected in series; each one receives an input signal and produces an output signal. They are connected such that the first amplifier's output leads to the second amplifier's input, the second amplifier's output leads to the third amplifier's input, and so on. The first amplifier's input value is 0, and the last amplifier's output leads to your ship's thrusters.
```
    O-------O  O-------O  O-------O  O-------O  O-------O
0 ->| Amp A |->| Amp B |->| Amp C |->| Amp D |->| Amp E |-> (to thrusters)
    O-------O  O-------O  O-------O  O-------O  O-------O
```
    
The Elves have sent you some Amplifier Controller Software (your puzzle input), a program that should run on your existing Intcode computer. Each amplifier will need to run a copy of the program.

When a copy of the program starts running on an amplifier, it will first use an input instruction to ask the amplifier for its current phase setting (an integer from 0 to 4). Each phase setting is used exactly once, but the Elves can't remember which amplifier needs which phase setting.

The program will then call another input instruction to get the amplifier's input signal, compute the correct output signal, and supply it back to the amplifier with an output instruction. (If the amplifier has not yet received an input signal, it waits until one arrives.)

Your job is to find the largest output signal that can be sent to the thrusters by trying every possible combination of phase settings on the amplifiers. Make sure that memory is not shared or reused between copies of the program.

In [7]:
%load_ext autoreload
%autoreload 2

In [22]:
from advent import run, load_data
from itertools import permutations

In [29]:
raw_data = load_data(7, intcode=True)
raw_data[:8]

[3, 8, 1001, 8, 10, 8, 105, 1]

In [57]:
sequences = permutations(range(5), 5)


In [32]:
run(raw_data);

Get this party started with input: 0
Get this party started with input: 0


14


In [49]:
def get_in(query='>'):
    return input(query)

def give_out(incoming):
    print(incoming)

In [50]:
from io import StringIO
import sys

first = get_in()
sys.stdin = StringIO(give_out('test_case'))
print('final out >>>', first)

> hi


test_case
final out >>> hi


In [41]:
get_in()
give_out('test_case')

 test_case


test_case


In [ ]:
input()

In [60]:
class Computer():
    
    # Intcode Computer Codes
    HALT = 99
    ADD = 1
    MULT = 2
    GET_IN = 3
    PRINT_OUT = 4
    JUMP_TRUE = 5
    JUMP_FALSE = 6
    LESS_THAN = 7
    EQUAL = 8
    
    # Saved States
    
    _input = None
    printout = None
    
    # Methods
    
    def opcode(self, raw_code):
        """Convert an opcode: ABCDE into {code, param_count, [modes]}

        Will auto-adjust the param_count and length of modes based
        on the dictionary param_counts

        ex) ABCDE  --  if DE is either 01 or 02:
        {
        'code': DE, 
        'param_count': 3, 
        'modes': [C, B, A],
        }
        """
        filled_code = str(raw_code).zfill(5)

        param_counts = {
            1: 3, 
            2: 3, 
            3: 1, 
            4: 1, 
            5: 2,
            6: 2,
            7: 3,
            8: 3,
            99: 0
        }

        # split @ 2 from right side, then reverse everything left of split
        modes = filled_code[:-2][::-1]
        code = int(filled_code[-2:])
        count = param_counts[code]

        modes = [int(x) for x in modes[:count]] 

        return {'code': code, 'param_count': count, 'modes': modes}
    
    def modeify(self, intcode, i):
        """Apply a mode to a parameter"""
        j = i + 1
        _opcode = opcode(intcode[i])
        params = intcode[j: j + _opcode['param_count']]
        modes = _opcode['modes']

        mode_covert = {
            0: lambda x: intcode[x],    # position mode
            1: lambda x: x              # immediate mode
        }

        output = [mode_covert[mode](param) for mode, param in zip(modes, params)]
        return output
    
    
    def get_new_pos(self, intcode, mode, i, delta_i):
        if mode == 0:
            return intcode[i + delta_i]
        elif mode == 1:
            return i + delta_i
        else:
            print(f'Mode not recognized. {mode} is not in [0, 1].')
            raise ValueError
    
    
    def single_code(self, intcode, i, halt=False, verbose=False):
        """Process a single intstruction from intcode

        return the same format we started with:
        (intcode, i, halt_status)
        """    
        _opcode = opcode(intcode[i])
        code = _opcode['code']
        modes = _opcode['modes']
        args = modeify(intcode, i)

        if verbose:
            print('params', intcode[i+1: i+1+_opcode['param_count']])
            print('modes', ['pos' if x is 0 else 'imm' for x in _opcode['modes']])
            print('args', args)

        if code == HALT:
            return intcode, i, True
        elif code == ADD:
            new_pos = get_new_pos(intcode, modes[-1], i, 3)
            new_value = sum(args[:-1])
        elif code == MULT:
            new_pos = get_new_pos(intcode, modes[-1], i, 3)
            new_value = args[0] * args[1]
        elif code == GET_IN:
            new_pos = get_new_pos(intcode, modes[-1], i, 1)
            new_value = int(input('>>'))
            self._input = new_value
        elif code == PRINT_OUT:
            new_pos = get_new_pos(intcode, modes[-1], i, 1)
            printout = intcode[new_pos]
            self.printout = printout
            print(printout)
        elif code == JUMP_TRUE:
            if args[0]:
                new_pointer = args[1]
            else:
                new_pointer = i + 3
        elif code == JUMP_FALSE:
            if args[0] == 0:
                new_pointer = args[1]
            else:
                new_pointer = i + 3
        elif code == LESS_THAN:
            if args[0] < args[1]:
                new_value = 1
            else:
                new_value = 0
            new_pos = get_new_pos(intcode, modes[-1], i, 3)
        elif code == EQUAL:
            if args[0] == args[1]:
                new_value = 1
            else:
                new_value = 0
            new_pos = get_new_pos(intcode, modes[-1], i, 3)

        if code not in [PRINT_OUT, JUMP_TRUE, JUMP_FALSE]:
            intcode[new_pos] = new_value
            if verbose:
                print(f'updating intcode[{new_pos}] to {intcode[new_pos]}')

        if code not in [JUMP_TRUE, JUMP_FALSE]:
            i += _opcode['param_count'] + 1
        else:
            i = new_pointer

        return intcode, i, halt

        

In [61]:
a = Computer()
a.opcode('10102')

{'code': 2, 'param_count': 3, 'modes': [1, 0, 1]}